In [1]:
import os
import streamlit as st
import pickle
import time
import langchain
from langchain import OpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

In [ ]:
os.environ["GOOGLE_API_KEY"] = "Uk"

In [59]:
llm=ChatGoogleGenerativeAI(model='gemini-2.0-flash',temperature=.9)

In [60]:
loaders=UnstructuredURLLoader(urls=[
    "https://www.pcworld.com/article/2604988/despite-assurances-nvidia-rtx-5090-is-melting-power-plugs-after-all.html",
    "https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html",
    "https://www.euronews.com/business/2025/02/12/deepseek-byd-collaboration-sends-ev-makers-shares-to-record-high"
])

In [61]:
data=loaders.load()

In [62]:
len(data)

3

In [63]:
data[0]

Document(metadata={'source': 'https://www.pcworld.com/article/2604988/despite-assurances-nvidia-rtx-5090-is-melting-power-plugs-after-all.html'}, page_content="Home / News / PCs & Components News\n\nNews\n\nDespite assurances, Nvidia’s RTX 5090 is melting power plugs after all\n\nOne Reddit user's RTX 5090 graphics card generated excessive heat and melted his power cable on both ends.\n\nBy Laura Pippig and Joel Lee\n\nFeb 11, 2025 8:17 am PST\n\nImage: Adam Patrick Murray / Foundry\n\nNvidia’s latest graphics cards sold out almost immediately, which means tons of people are still clamoring to get their hands on one. But with recent issues rearing their heads, you might want to hold off.\n\nLast week, RTX 5090 cards started dying after Nvidia’s driver update. More recently, one Reddit user named /u/ivan6953 reported in the Nvidia subreddit that the power connector on his RTX 5090 had melted while playing Battlefield V.\n\n“I am not distant from the PC-building world and know what I’m d

In [64]:
doc_splitter=RecursiveCharacterTextSplitter(chunk_size=520,chunk_overlap=100)

In [65]:
doc_chunks=doc_splitter.split_documents(data)

In [66]:
len(doc_chunks)

32

In [67]:
embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [ ]:
vectorizeindex_gemini=FAISS.from_documents(doc_chunks,embedding)

In [69]:
vectorizeindex_gemini

In [ ]:
# file_path="vector_index.pkl"
# with open(file_path,"wb") as f:
#     pickle.dump(vectorizeindex_gemini,f)

TypeError: no default __reduce__ due to non-trivial __cinit__

In [70]:
file_path="vector_index.pkl"
vectorizeindex_gemini.save_local(file_path)

In [71]:
vectorizeindex_gemini = FAISS.load_local(file_path, embedding,allow_dangerous_deserialization=True)

In [72]:
from langchain.chains import RetrievalQAWithSourcesChain

In [73]:
chain = RetrievalQAWithSourcesChain.from_llm(llm=llm,retriever=vectorizeindex_gemini.as_retriever())

In [74]:
chain

RetrievalQAWithSourcesChain(verbose=False, combine_documents_chain=MapReduceDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Use the following portion of a long document to see if any of the text is relevant to answer the question. \nReturn any relevant text verbatim.\n{context}\nQuestion: {question}\nRelevant text, if any:'), llm=ChatGoogleGenerativeAI(model='models/gemini-2.0-flash', google_api_key=SecretStr('**********'), temperature=0.9, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x0000024481CC37D0>, default_metadata=()), output_parser=StrOutputParser(), llm_kwargs={}), reduce_documents_chain=ReduceDocumentsChain(verbose=False, combine_documents_chain=StuffDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question', 'summaries'], input_

In [75]:
query = "who is the chinese ev maker?"

langchain.debug = True

chain({"question":query},return_only_outputs=True)

[chain/start] [chain:RetrievalQAWithSourcesChain] Entering Chain run with input:
{
  "question": "who is the chinese ev maker?"
}
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "input_list": [
    {
      "context": "The Chinese EV maker, Tesla's key competitor, said it would roll out a self-driving system in all of its models, ranging from cheaper to more exclusive vehicles.\n\nBYD founder Wang Chuanfu said on Monday that his firm sought to make autonomous driving less of a luxury and more of a commonplace safety feature.\n\nThis came after the Chinese firm had previously taken more of a cautious approach to autonomous driving compared to competitors.",
      "question": "who is the chinese ev maker?"
    },
    {
      "context": "While this challenged assumptions about AI, San 

{'answer': 'BYD is a Chinese EV maker.\n',
 'sources': 'https://www.euronews.com/business/2025/02/12/deepseek-byd-collaboration-sends-ev-makers-shares-to-record-high'}

In [76]:
data[1]

Document(metadata={'source': 'https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html'}, page_content='English\n\nHindi\n\nGujarati\n\nSpecials\n\nHello, Login\n\nHello, Login\n\nLog-inor Sign-Up\n\nMy Account\n\nMy Profile\n\nMy Portfolio\n\nMy Watchlist\n\nMy Alerts\n\nMy Messages\n\nPrice Alerts\n\nMy Profile\n\nMy PRO\n\nMy Portfolio\n\nMy Watchlist\n\nMy Alerts\n\nMy Messages\n\nPrice Alerts\n\nLogout\n\nLoans up to ₹50 LAKHS\n\nFixed Deposits\n\nCredit CardsLifetime Free\n\nCredit Score\n\nChat with Us\n\nDownload App\n\nFollow us on:\n\nGo Ad-Free\n\nMy Alerts\n\nco-presented by\n\nassociated by\n\nBusiness\n\nMarkets\n\nStocks\n\nEconomy\n\nCompanies\n\nTrends\n\nIPO\n\nOpinion\n\nEV Special\n\nOptions FestWebinar\n\nHomeNewsBusinessTata Motors launches Punch iCNG, price starts at Rs 7.1 lakh\n\nTrending Topics\n\nNew Income tax bill Vodafone Idea newsIPO ListingsRBI Repo Rate CutIndia GDP\n\nTata Motors launches Punc

In [77]:
query="Who released a video about the RTX 5090 incident ?"

langchain.debug=True

chain({"question":query},return_only_outputs=True)

[chain/start] [chain:RetrievalQAWithSourcesChain] Entering Chain run with input:
{
  "question": "Who released a video about the RTX 5090 incident ?"
}
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "input_list": [
    {
      "context": "A detailed investigation is needed\n\nHardware YouTuber Roman “der8auer” Hartung has released a video about this incident. He contacted Ivan and examined his RTX 5090 and the affected cable more closely. Under the microscope, he could see clear evidence of overheating, and the plastic around the pins had also broken off when the cable was pulled out. Apparently, two of the 12VHPWR pins (out of the total six used in the RTX 5090) were affected.",
      "question": "Who released a video about the RTX 5090 incident ?"
    },
    {
      "context": "

{'answer': 'Hardware YouTuber Roman “der8auer” Hartung released a video about the RTX 5090 incident.\n',
 'sources': 'https://www.pcworld.com/article/2604988/despite-assurances-nvidia-rtx-5090-is-melting-power-plugs-after-all.html'}

In [78]:
query="Who owns rtx5090?"

langchain.debug=True

chain({"question":query},return_only_outputs=True)

[chain/start] [chain:RetrievalQAWithSourcesChain] Entering Chain run with input:
{
  "question": "Who owns rtx5090?"
}
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "input_list": [
    {
      "context": "As of now, the only thing anyone can do is to send any affected graphics cards to Nvidia so that the problem can be investigated in more detail. No other cases are currently known, but that’s likely due to the general lack of availability of the RTX 5090 thus far.\n\nFurther reading: Nvidia’s RTX 5090 is a brutally fast game-changer\n\nThis article originally appeared on our sister publication PC-WELT and was translated and localized from German.",
      "question": "Who owns rtx5090?"
    },
    {
      "context": "Home / News / PCs & Components News\n\nNews\n\nDespite assuranc

{'answer': 'Nvidia owns the RTX 5090.\n',
 'sources': 'https://www.pcworld.com/article/2604988/despite-assurances-nvidia-rtx-5090-is-melting-power-plugs-after-all.html'}

In [54]:
query="What did Roman Hartung observe under the microscope?"

langchain.debug=True

chain({"question":query},return_only_outputs=True)

[chain/start] [chain:RetrievalQAWithSourcesChain] Entering Chain run with input:
{
  "question": "What did Roman Hartung observe under the microscope?"
}
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "input_list": [
    {
      "context": "Controversy with self-driving system\n\nWhile Tesla maintains their status as a pioneer in the automobile industry, their self-driving system has faced significant backlash over the years. In October 2024, it was reported that US authorities have opened up an investigation into Tesla’s self-driving system which has resulted in numerous crashes, including one pedestrian death. The investigation could lead to the National Highway Traffic Safety Administration issuing a recall of the vehicles if the agency finds they pose an unreasonable risk to s

{'answer': 'The provided context does not mention Roman Hartung or his observations, so I cannot answer this question from the provided context.\n',
 'sources': 'https://www.ecoticias.com/en/tesla-delays-its-futuristic-project/11241/, https://www.pcworld.com/article/2604988/despite-assurances-nvidia-rtx-5090-is-melting-power-plugs-after-all.html, https://www.pcworld.com/article/2604988/despite-assurances-nvidia-rtx-5090-is-melting-power-plugs-after-all.html'}

In [79]:
query="Who owns rtx5090?"

langchain.debug=True

chain({"question":query},return_only_outputs=True)

[chain/start] [chain:RetrievalQAWithSourcesChain] Entering Chain run with input:
{
  "question": "Who owns rtx5090?"
}
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "input_list": [
    {
      "context": "As of now, the only thing anyone can do is to send any affected graphics cards to Nvidia so that the problem can be investigated in more detail. No other cases are currently known, but that’s likely due to the general lack of availability of the RTX 5090 thus far.\n\nFurther reading: Nvidia’s RTX 5090 is a brutally fast game-changer\n\nThis article originally appeared on our sister publication PC-WELT and was translated and localized from German.",
      "question": "Who owns rtx5090?"
    },
    {
      "context": "Home / News / PCs & Components News\n\nNews\n\nDespite assuranc

{'answer': 'Nvidia owns the RTX 5090.\n',
 'sources': 'https://www.pcworld.com/article/2604988/despite-assurances-nvidia-rtx-5090-is-melting-power-plugs-after-all.html'}

In [80]:
query="cards started dying after Nvidia’s driver update?"

langchain.debug=True

chain({"question":query},return_only_outputs=True)

[chain/start] [chain:RetrievalQAWithSourcesChain] Entering Chain run with input:
{
  "question": "cards started dying after Nvidia’s driver update?"
}
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "input_list": [
    {
      "context": "By Laura Pippig and Joel Lee\n\nFeb 11, 2025 8:17 am PST\n\nImage: Adam Patrick Murray / Foundry\n\nNvidia’s latest graphics cards sold out almost immediately, which means tons of people are still clamoring to get their hands on one. But with recent issues rearing their heads, you might want to hold off.",
      "question": "cards started dying after Nvidia’s driver update?"
    },
    {
      "context": "What makes this development interesting is that Nvidia said just last month that these new RTX 50-series cards won’t melt power plugs this time.

{'answer': 'RTX 5090 cards started dying after Nvidia’s driver update.\n',
 'sources': 'https://www.pcworld.com/article/2604988/despite-assurances-nvidia-rtx-5090-is-melting-power-plugs-after-all.html'}